In [3]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import  cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [6]:
ls

data/  data.men_shoes.csv  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [7]:
ls data

men_shoes.csv  shoes_prices.cvs


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [17]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)


In [18]:
run_model(['brand_cat'], model)


(-57.31783843165656, 4.181246596160967)

In [21]:
test = {'key': 'value'}
test['key']

str(test)

"{'key': 'value'}"

In [23]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [24]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [25]:
literal_eval(str_dict)[0]['value']

['Men']

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  return output_dict
df['features_parsed'] = df['features'].map(parse_features)

In [39]:
df['features_parsed'].head()

0    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2    {'gender': 'men', 'color': 'black', 'shipping ...
3    {'gender': 'men', 'color': 'black', 'shipping ...
4    {'gender': 'men', 'color': 'black', 'shipping ...
Name: features_parsed, dtype: object

In [40]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

476

In [46]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [47]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_number of items', 'feat_hat style', 'feat_fine or fashion',
       'feat_neckline', 'feat_crystal', 'feat_picture', 'feat_polarized',
       'feat_date first available', 'feat_watch power source',
       'feat_weather-resistant'],
      dtype='object', length=526)

In [48]:
df.shape[0]

18280

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] *100

In [53]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [65]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8846, 531)

In [84]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_multi pack indicator',
 'feat_certifications and listings',
 'feat_catalog',
 'feat_fabrication',
 'feat_recommended location',
 'feat_clothing category',
 'feat_location - country',
 'feat_location - city/state',
 'feat_shoe category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_closure_cat',
 'feat_number of pockets_cat',
 'feat_clothing size type_cat',
 'feat_case type_cat',
 'feat_fits models_cat',
 'feat_date first available at amazon.co.uk_cat',
 'feat_machine washable_cat',
 'feat_sku#_cat',
 'feat_compass_cat',
 'feat_pattern or design_cat',
 'feat_ean_cat',
 'feat_resizable_cat',
 'feat_removable hood_cat',
 'feat_manufacturer sku_cat',
 'feat_item spec attr - style (ebay)_cat',
 'feat_shape_cat',
 'feat_eye size:_cat',
 'feat_number of compartments_cat',
 'feat_case material_cat',
 'feat_model no._cat',
 'feat_dimensions_ca

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat', 'feat_metal type_cat', 'feat_ring size_cat', 'feat_shape_cat', 'feat_occasion_cat']
#feats += feats_cat
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [94]:
X = df[feats].values
y = df['prices_amountmin'].values

m= RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)
print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.25396116690338, 4.282957218599908)


Weight,Feature
0.2589 ± 0.0071,brand_cat
0.1028 ± 0.0086,feat_material_cat
0.0222 ± 0.0030,feat_gender_cat
0.0169 ± 0.0007,feat_brand_cat
0.0130 ± 0.0012,feat_shape_cat
0.0057 ± 0.0010,feat_metal type_cat
0.0032 ± 0.0009,feat_ring size_cat
0.0023 ± 0.0004,feat_occasion_cat
0.0020 ± 0.0007,feat_style_cat
0.0001 ± 0.0000,feat_sport_cat


In [74]:
df['brand'].value_counts(normalize=True)

nike                     0.097210
puma                     0.033315
ralph lauren             0.028775
vans                     0.021116
new balance              0.020295
                           ...   
u.s. polo association    0.000055
ralph lauren rrl         0.000055
rush by gordon rush      0.000055
customised_perfection    0.000055
mancini leather          0.000055
Name: brand, Length: 1732, dtype: float64

In [96]:
ls

data/  data.men_shoes.csv  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [100]:
!git add dw_matrix/matrix_one/day5.ipynb

fatal: pathspec 'dw_matrix/matrix_one/day5.ipynb' did not match any files


In [101]:
!git add day5.ipynb

fatal: pathspec 'day5.ipynb' did not match any files
